In [1]:
import pandas as pd
from pandas_datareader import data, wb
from datetime import datetime
import numpy as np
import graphviz
import sys
import os

In [2]:
if not os.path.exists('GOOG.csv'):
    df = data.DataReader('GOOG', 'yahoo', datetime(2010, 1, 1), datetime(2017, 11, 1))
    df.to_csv('GOOG.csv')
else:
    df = pd.read_csv('GOOG.csv')

In [3]:
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,311.449310,312.721039,310.103088,311.349976,311.349976,3937800
1,2010-01-05,311.563568,311.891449,308.761810,309.978882,309.978882,6048500
2,2010-01-06,310.907837,310.907837,301.220856,302.164703,302.164703,8009000
3,2010-01-07,302.731018,303.029083,294.410156,295.130463,295.130463,12912000
4,2010-01-08,294.087250,299.675903,292.651581,299.064880,299.064880,9509900
5,2010-01-11,300.276978,300.276978,295.100647,298.612823,298.612823,14519600
6,2010-01-12,296.893982,297.147339,292.100159,293.332153,293.332153,9769600
7,2010-01-13,286.382355,292.288940,285.095734,291.648102,291.648102,13077600
8,2010-01-14,290.063416,295.180145,289.521942,293.019196,293.019196,8535300
9,2010-01-15,294.752930,294.862213,287.152344,288.126007,288.126007,10939600


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1973 entries, 0 to 1972
Data columns (total 7 columns):
Date         1973 non-null object
Open         1973 non-null float64
High         1973 non-null float64
Low          1973 non-null float64
Close        1973 non-null float64
Adj Close    1973 non-null float64
Volume       1973 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 108.0+ KB


In [5]:
df.drop('Date', axis=1, inplace=True, errors='ignore')
df = df.diff()
df.iloc[0, :] = 0
df.head()

,Open,High,Low,Close,Adj Close,Volume
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.114258,-0.829590,-1.341278,-1.371094,-1.371094,2110700.0
2,-0.655731,-0.983612,-7.540954,-7.814179,-7.814179,1960500.0
3,-8.176819,-7.878754,-6.810700,-7.034240,-7.034240,4903000.0
4,-8.643768,-3.353180,-1.758575,3.934417,3.934417,-3402100.0


In [6]:
df = df[['Open', 'High', 'Low', 'Close']]

In [7]:
df.describe()

,Open,High,Low,Close
count,1973.000000,1973.000000,1973.000000,1973.000000
mean,0.357709,0.363380,0.358260,0.361961
std,7.684202,7.012193,7.073283,7.497264
min,-66.780029,-47.500000,-47.280029,-40.369995
25%,-3.039978,-2.434173,-2.569947,-2.911042
50%,0.293091,0.239990,0.499695,0.168914
75%,3.859986,3.020020,3.530030,3.669983
max,83.880005,93.788025,80.000000,93.080017


In [9]:
n_train = int(2/3*df.shape[0])
Xtr, Xte = df.iloc[:n_train, :], df.iloc[n_train:,:]
N_FEATS = Xtr.shape[1]
N_FEATS

4

In [10]:
WND_DAYS = 7
MAX_NORM = 64

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [12]:
class RNN1(nn.Module):
    
    
    def __init__(self, n_in, n_mem, n_out):
        super(RNN1, self).__init__()
        self.n_in, self.n_mem, self.n_out = n_in, n_mem, n_out
        self.forget = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.learn_new = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.save_new = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.output = nn.Linear(in_features=(n_in + n_mem), out_features=n_out)
        
    def init_weights(self, var=.1):
        self.forget.weight.data.uniform_(-var, var)
        self.learn_new.weight.data.uniform_(-var, var)
        self.save_new.weight.data.uniform_(-var, var)
        self.output.weight.data.uniform_(-var, var)
    
    def forward(self, x, mem):
        # print(x.data.shape, mem.data.shape)
        xmem = torch.cat((x, mem))
        forget = F.sigmoid(self.forget(xmem))
        mem = mem*forget
        new_mem = F.tanh(self.learn_new(xmem))
        mem_mask = F.sigmoid(self.save_new(xmem))
        mem = mem + new_mem*mem_mask
        xmem = torch.cat((x, mem))
        out = self.output(xmem)
        return (out, mem)
    
    def init_mem(self):
        return Variable(torch.zeros(self.n_mem))

In [13]:
rnn1 = RNN1(n_in=WND_DAYS*N_FEATS, n_mem=48, n_out=WND_DAYS)

In [19]:
def loss(
    model,
    X,
    criterion,
    wnd=WND_DAYS,
    var='Close',
    lr=None,
    ):
    mem = model.init_mem()
    total_L = 0
    for i in range(wnd, X.shape[0]):
        model.zero_grad()
        x = X.iloc[i-wnd+1:i+1,:].as_matrix().reshape(-1)
        x = torch.Tensor(x)
        x = Variable(x)
        y = torch.Tensor(X.iloc[i:i+wnd,:].loc[:, var].as_matrix().reshape(-1))
        y = Variable(y)
        y_pred, mem = model(x, mem)
        y_pred = y_pred[:y.data.shape[0]]
        L = criterion(y, y_pred)
        total_L += L.data
        
        if lr is not None:
            L.backward(retain_graph=1)
            torch.nn.utils.clip_grad_norm(model.parameters(), MAX_NORM)
            for p in model.parameters():
                p.data.add_(-lr, p.grad.data)
    return total_L/X.shape[0]

In [15]:
def objective(y_pred, y):
    L = -y_pred*y
    L[L < 0] = 0
    L = L.sum()
    L2 = y_pred - y
    L2 = (L2*L2).sum()/sum(y.data.shape)
    return L
criterion = objective

In [20]:
loss(rnn1, Xtr, criterion, lr=.01)


 67.7105
[torch.FloatTensor of size 1]

In [21]:
for i in range(20):
    print('Epoch #%s; Loss: %s' % (i, loss(rnn1, Xtr, criterion)))

Epoch #0; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #1; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #2; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #3; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #4; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #5; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #6; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #7; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #8; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #9; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #10; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #11; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #12; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #13; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #14; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #15; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #16; Loss: 
 53.7688
[torch.FloatTensor of size 1]

Epoch #17; Loss: 
 53.76

In [22]:
rnn1

RNN1 (
  (forget): Linear (76 -> 48)
  (learn_new): Linear (76 -> 48)
  (save_new): Linear (76 -> 48)
  (output): Linear (76 -> 7)
)

In [23]:
#list(rnn1.parameters())

In [24]:
torch.save(rnn1.state_dict(), 'rnn1.weights')

In [ ]:
loss(rnn1, Xte, lr=None)

In [25]:
loss(rnn1, X, lr=None)

NameError: name 'X' is not defined

In [ ]:
def predict(model, X, wnd=WND_DAYS):
    mem = model.init_mem()
    for i in range(wnd-1, X.shape[0]):
        x = X.iloc[i-wnd+1:i+1, :].as_matrix().reshape(-1)
        x = torch.Tensor(x)
        x = Variable(x)
        y, mem = model(x, mem)
    return y

In [ ]:
predict(rnn1, Xte.iloc[:128, :]).data

In [ ]:
Xte.iloc[128:128+WND_DAYS,:]['Close']